In [1]:
#r "/home/jovyan/Notebooks/extensions/Microsoft.DotNet.Interactive.XPlot/Microsoft.DotNet.Interactive.XPlot.dll"
#extend "/home/jovyan/Notebooks/extensions/Microsoft.DotNet.Interactive.XPlot/Microsoft.DotNet.Interactive.XPlot.dll"
#r "nuget:Microsoft.ML.FastTree,1.4.0-preview2"

using Microsoft.DotNet.Interactive.XPlot;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers.FastTree;

Loading kernel extension Microsoft.DotNet.Interactive.XPlot.MlKernelExtension from assembly F:\git\DotNetInteractiveExtension\src\Microsoft.DotNet.Interactive.XPlot\bin\Debug\netcoreapp3.0\Microsoft.DotNet.Interactive.XPlot.dll

Loaded kernel extension Microsoft.DotNet.Interactive.XPlot.MlKernelExtension from assembly F:\git\DotNetInteractiveExtension\src\Microsoft.DotNet.Interactive.XPlot\bin\Debug\netcoreapp3.0\Microsoft.DotNet.Interactive.XPlot.dll

Installing package Microsoft.ML.FastTree, version 1.4.0-preview2............done!

Successfully added reference to package Microsoft.ML.FastTree, version 1.4.0-preview2

# Formatting code for FastTree models
The following code takes a FastTree regression tree and turns it into a DecisionTreeData, which can be visualized by the Microsoft.DotNet.Interactive.XPlot extension.

In [2]:
private DecisionTreeData DisplayTree(RegressionTreeEnsemble ensemble, in VBuffer<ReadOnlyMemory<char>> featureNames = default)
{
    // just get the first tree, for now
    return DisplayTree(ensemble.Trees.FirstOrDefault(), featureNames);
}

private DecisionTreeData DisplayTree(RegressionTree tree, in VBuffer<ReadOnlyMemory<char>> featureNames = default)
{
    DecisionTreeData treeData = new DecisionTreeData();

    if (tree is object)
    {
        NodeData[] nodes = new NodeData[tree.NumberOfNodes];
        StringBuilder labelBuilder = new StringBuilder();
        for (int node = 0; node < tree.NumberOfNodes; node++)
        {
            nodes[node] = new NodeData();
            int featureIndex = tree.NumericalSplitFeatureIndexes[node];
            float splitThreshold = tree.NumericalSplitThresholds[node];
            
            ReadOnlyMemory<char> featureName = featureNames.GetItemOrDefault(featureIndex);
            if (!featureName.IsEmpty)
            {
                labelBuilder.Append(featureName);
            }
            else
            {
                labelBuilder.Append('f');
                labelBuilder.Append(featureIndex);
            }
            labelBuilder.Append($" > ");
            labelBuilder.Append(splitThreshold.ToString("n2"));

            nodes[node].Label = labelBuilder.ToString();
            labelBuilder.Clear();
        }

        NodeData[] leaves = new NodeData[tree.NumberOfLeaves];
        for (int leaf = 0; leaf < tree.NumberOfLeaves; leaf++)
        {
            leaves[leaf] = new NodeData();
            leaves[leaf].Label = tree.LeafValues[leaf].ToString("n2");
        }

        NodeData GetNodeData(int child)
        {
            return child >= 0
                ? nodes[child]
                : leaves[~child];
        }

        // hook the nodes up
        for (int node = 0; node < tree.NumberOfNodes; node++)
        {
            // the RightChild is the 'greater than' path, so put that first
            nodes[node].Children.Add(GetNodeData(tree.RightChild[node]));
            nodes[node].Children.Add(GetNodeData(tree.LeftChild[node]));
        }

        if (nodes.Length > 0)
        {
            treeData.Root = nodes[0];
        }
    }

    return treeData;
}

# Load Model
Load the model from disk. Get the FastTree transformer from it. And get the names of each feature column.

In [13]:
MLContext mlContext = new MLContext();
ITransformer model = mlContext.Model.Load("HousingTreeModel.zip", out DataViewSchema inputSchema);

DataViewSchema outputSchema = model.GetOutputSchema(inputSchema);

ISingleFeaturePredictionTransformer<TreeEnsembleModelParametersBasedOnRegressionTree> predictor = ((TransformerChain<ITransformer>)model).LastTransformer as ISingleFeaturePredictionTransformer<TreeEnsembleModelParametersBasedOnRegressionTree>;
VBuffer<ReadOnlyMemory<char>> featureNames = default;
outputSchema[predictor.FeatureColumnName].GetSlotNames(ref featureNames);

# Display the Tree

In [14]:
DisplayTree(predictor.Model.TrainedTreeEnsemble, featureNames)